## Feature Selection

In [5]:
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler

In [8]:
# Import dataframe 

df = pd.read_csv('/Users/gabeheim/documents/repos/bender/source files/cap1_df.csv', index_col='time')

In [9]:
# Standardize the data

sc_x = StandardScaler()
df_scaled = pd.DataFrame(sc_x.fit_transform(df), index=df.index, columns=df.columns)

In [10]:
# Perfrom differencing to stationalize the series

# Creat difference function, with default value of lag 24
def difference(dataset, interval=24):
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        diff.append(value)
    return diff

# Differencing the dataset
inter_d = 24
df_diff = pd.DataFrame(index=df.index)
for i in df.columns:
    data = difference(df_scaled[i], inter_d)
    data = pd.Series(np.append(np.repeat(np.nan, inter_d), data), index=df.index, name=i)
    df_diff = pd.concat([df_diff, data], ignore_index=False, axis=1)


In [11]:
# Perform Dickey-fuller test to test the differenced series for stationality

diff = df_diff.dropna()

for i in diff.columns: 
    x = diff[i].values
    result = adfuller(x)
    print('\033[1m' + i + '\033[0m')
    print('ADF Statistic:', result[0])
    print('p-value:', result[1])

XEM
ADF Statistic: -6.8347645152100895
p-value: 1.8546167699788319e-09
IOT
ADF Statistic: -5.9849987706163965
p-value: 1.8009544435952284e-07
amd
ADF Statistic: -5.020269791994407
p-value: 2.023192354839968e-05
nvda
ADF Statistic: -5.495891001501147
p-value: 2.1239443581496083e-06
mc_iota
ADF Statistic: -5.684332592443915
p-value: 8.346295935374271e-07
mc_nem
ADF Statistic: -5.9125369923810025
p-value: 2.6182678885683667e-07
gg_crypto
ADF Statistic: -5.644299288447219
p-value: 1.0196718079380073e-06
gg_nem_p
ADF Statistic: -6.939243555123052
p-value: 1.0353170023035572e-09
gg_iota_p
ADF Statistic: -6.958625671192854
p-value: 9.288088478212726e-10
gg_bitcoin_p
ADF Statistic: -6.0930336491202235
p-value: 1.0255751681973199e-07
gg_nem_w
ADF Statistic: -7.028064921450798
p-value: 6.288873084549656e-10
gg_iota_w
ADF Statistic: -6.465079412079976
p-value: 1.4118279708128427e-08
gg_gpu
ADF Statistic: -5.408638494228449
p-value: 3.248531026331344e-06
rd_CrypCurr_g
ADF Statistic: -6.16375948433

In [14]:
# Splitting data to test/train sets

X = diff[['IOT', 'amd', 'nvda', 'gg_crypto','gg_nem_p', 'gg_iota_p', 'gg_bitcoin_p', 'gg_nem_w', 'gg_iota_w','gg_gpu', 'rd_CrypCurr_g', 'rd_nem_g', 'rd_Iota_g']]
#not using marketcap data!!
Ytrain = pd.DataFrame(diff[:1310]['XEM'])
Ytest = pd.DataFrame(diff[1310:]['XEM'])

In [15]:
# Performing ElasticNet Regression 

en = ElasticNet(alpha=0.1, normalize=False)
en.fit(X[:1310], Ytrain)
coef = list(en.coef_)
count = 0
for i in X.columns:
    print(i, ':', coef[count])
    count = count + 1

IOT : 0.22404955263827642
amd : 0.0
nvda : 0.0
gg_crypto : -0.034198443750487736
gg_nem_p : 0.0
gg_iota_p : -0.0
gg_bitcoin_p : -0.0
gg_nem_w : 0.04550451889245516
gg_iota_w : -0.0
gg_gpu : 0.0
rd_CrypCurr_g : -0.0
rd_nem_g : 0.0
rd_Iota_g : 0.0


In [16]:
# Based on the ElasticNet results, manually identifying selected features (features with non-zero coefficients)

sig = ['IOT', 'gg_crypto', 'gg_nem_w']
#sig = ['IOT', 'amd', 'nvda', 'gg_crypto','gg_nem_p', 'gg_iota_p', 'gg_bitcoin_p', 'gg_nem_w', 'gg_iota_w','gg_gpu', 'rd_CrypCurr_g', 'rd_nem_g', 'rd_Iota_g']

In [17]:
# Constructing dataframe

X_sig = pd.DataFrame(index=X.index, columns=[sig])
for i in sig:
    X_sig[i] = X[i]
    count = count + 1

# Save dataframe
X_sig[:1310].to_csv('X_train.csv')
X_sig[1310:].to_csv('X_test.csv')
Ytrain.to_csv('Y_train.csv')
Ytest.to_csv('Y_test.csv')

### Conclusions: 
We performed ElasticNet regularization on all 13 predictors. Then, we selected 3 predictors with non-zero coefficents for our model. 